In [6]:
search_for = 104
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.013669013977050781
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 365444151
type: [1, 1, 1, 1, 104] 104
cases of this type: 116985856
10000 62.44324337370284
20000 68.4739382802367
30000 69.49040857750835
40000 259.0788337305826
50000 286.5355882913377
60000 262.0468807007722
70000 266.9866146597117
80000 259.0848497689293
90000 255.0872124566797
100000 251.90814834115608
110000 252.02356928306293
120000 266.7730573986019
130000 257.80353916306456
140000 260.5118691242666
150000 254.9316308383171
160000 252.95085891298797
170000 247.15878243161663
180000 250.34823518035185
190000 246.61586738927193
200000 250.5117154366504
210000 245.89492247483182
220000 244.98262625142593
230000 253.23916052885392
240000 261.57333571171176
250000 246.48034429819492
260000 243.3341105649554
270000 256.1609212786482
280000 265.3365543400778
290000 284.7357312186288
300000 279.54406961024085
310000 261.6661501

3060000 234.82188017046056
3070000 235.3083679975037
3080000 250.946155451754
3090000 246.00350863938098
3100000 252.21265848926828
3110000 247.66137158448123
3120000 285.3405477449702
3130000 238.7817474764333
3140000 238.55147589903186
3150000 252.687618113379
3160000 244.35671363202198
3170000 254.03027737906984
3180000 242.24730441441733
3190000 244.64082341972065
3200000 233.29905191526217
3210000 233.4248842036843
3220000 251.8532082008664
3230000 240.82207832980853
3240000 254.69253027564162
3250000 266.4185280851837
3260000 244.33883044247798
3270000 265.26511745232546
3280000 238.54403312790623
3290000 254.4603747082845
3300000 227.894476312069
3310000 232.04270629252358
3320000 269.7038003300665
3330000 269.8987091459354
3340000 100.48366698256366
3350000 97.15860527961844
3360000 79.75401867843976
3370000 80.62530481740225
3380000 198.68911108378137
3390000 204.42855295782834
3400000 72.04463084069454
3410000 64.5477004068557
3420000 64.49877844856276
3430000 64.411278122388

6150000 212.06145913885442
6160000 249.9260669162072
6170000 156.60936770669048
6180000 32.99740437736619
6190000 32.61727688306623
6200000 32.587549959222585
6210000 33.06359567868538
6220000 38.51142453568577
6230000 294.4529648596658
6240000 240.39421821044377
6250000 200.61325367365197
6260000 208.52257301767642
6270000 211.76644623240043
6280000 225.79152981298682
6290000 241.1222169887865
6300000 243.86999102090522
6310000 238.54094216912017
6320000 231.12724010029612
6330000 225.92570163067552
6340000 226.4119679988359
6350000 217.51640590569497
6360000 218.17972997059036
6370000 220.07140299446274
6380000 215.0613336326335
6390000 220.373026186983
6400000 216.73758683294687
6410000 226.3979007672165
6420000 215.15158957241704
6430000 232.88924572963225
6440000 229.5478433315533
6450000 224.78464369205022
6460000 241.56579918699697
6470000 245.47774168470312
6480000 220.92706307687507
6490000 243.16387402567253
6500000 242.67857318373075
6510000 244.42658838207404
6520000 231.88

9240000 228.21036609280725
9250000 230.91619500551639
9260000 218.34907548759594
9270000 226.3701646339767
9280000 215.8226176345364
9290000 230.6861143018658
9300000 217.8926410277311
9310000 220.39862403742998
9320000 251.27384122519834
9330000 229.38366827768363
9340000 234.61229837092216
9350000 235.9841188557835
9360000 216.37545001352075
9370000 233.68026451432723
9380000 229.15143665246825
9390000 226.77228984803875
9400000 230.5855488804211
9410000 221.8095917703283
9420000 216.22228412101217
9430000 223.9885736105325
9440000 224.89712074932092
9450000 218.6827814049182
9460000 213.5828204171039
9470000 221.78521110251344
9480000 228.4438452541463
9490000 226.00146696306473
9500000 213.57912727620453
9510000 197.5959878617954
9520000 194.91486259824177
9530000 217.4895718014224
9540000 262.02814366891226
9550000 32.87983055200213
9560000 32.718557831142135
9570000 32.749979323109585
9580000 32.78602538640317
9590000 32.899945122510886
9600000 177.6885113740561
9610000 264.13795

12240000 243.712540538996
12250000 147.77598935379152
12260000 80.47229775775799
12270000 81.46404132849723
12280000 80.28846539784799
12290000 79.43032582533233
12300000 116.22240614012861
12310000 276.86336239955807
12320000 261.87033743528264
12330000 221.47197505518832
12340000 233.79396393778515
12350000 226.6581378905641
12360000 228.70615032580108
12370000 256.74165339764215
12380000 250.71424003427992
12390000 235.35920210134265
12400000 244.27764574666045
12410000 239.9495631064726
12420000 227.04524705433673
12430000 223.89068430634597
12440000 223.96317448900876
12450000 235.51849791974175
12460000 241.53901936103804
12470000 276.7521231902603
12480000 80.1564737989431
12490000 63.46522146307846
12500000 63.64588288431723
12510000 64.09043938056466
12520000 64.09086026798528
12530000 175.61005551363377
12540000 254.65518524088864
12550000 227.6760602833829
12560000 217.00785020418954
12570000 227.21469494472328
12580000 235.57112426248128
12590000 228.99658210438625
12600000

...

354460000 0.4749654279849198
354470000 0.7664775218001736
354480000 0.5679907211425901
354490000 0.8445940474577083
354500000 0.49890901755113737
354510000 0.5207051813792387
354520000 0.6310807576055328
354530000 0.7188489192543494
354540000 0.48746364534656206
354550000 0.4846854634571738
354560000 0.4878632286464969
354570000 0.478346802615583
354580000 0.4795429179996716
354590000 0.48757853501229154
354600000 0.4849946800841888
354610000 0.47178904591564996
354620000 0.4687254466870427
354630000 0.47669829469656944
354640000 0.7131809876326852
354650000 0.5911494638482532
354660000 0.45753863960176705
354670000 0.565150042821149
354680000 0.467666337847544
354690000 0.47191841660656536
354700000 0.4668228785199126
354710000 0.47969406276544596
354720000 0.4796945522993803
354730000 0.4808192310254044
354740000 0.47915043025608856
354750000 0.4795423541469574
354760000 0.465999613195267
354770000 0.657625076416903
354780000 0.6521789517921011
354790000 0.535480177490857
354800000 0

357280000 0.8007684186482495
357290000 0.782168843561444
357300000 0.49784893413394693
357310000 0.46926583390978976
357320000 0.48500061500529446
357330000 0.42669100644836827
357340000 0.5511151047578653
357350000 1.0497103257444766
357360000 0.7229510161873698
357370000 0.8465171484430167
357380000 0.9091069887102975
357390000 1.229654964888215
357400000 1.4259742496198482
357410000 1.086416403434409
357420000 0.8478640101521016
357430000 0.6426043171580235
357440000 0.42994570535086923
357450000 0.3864968948994875
357460000 0.386747246511115
357470000 0.4074088969449997
357480000 0.4957797786348462
357490000 0.8163727034577795
357500000 0.7643213116625944
357510000 1.0273096494707465
357520000 1.242504923552076
357530000 0.7117605178035366
357540000 0.9564808846539258
357550000 0.8669718019853433
357560000 1.3126508765823577
357570000 0.8251633021341761
357580000 0.46973688455715445
357590000 0.4655169169905914
357600000 0.6620664866931636
357610000 0.6737991848082278
357620000 0.7

360010000 0.23617675825077955
360020000 0.3374865291733874
360030000 0.30735231747482217
360040000 0.23401820339194274
360050000 0.23534397140032715
360060000 0.3199896316160559
360070000 0.40090818294468855
360080000 0.22894160196883148
360090000 0.33876907971062264
360100000 0.24492512015683124
360110000 0.2311197572268115
360120000 0.23848830900237958
360130000 0.40928892090739144
360140000 0.4066420907004608
360150000 0.2948599345990419
360160000 0.4037391723240216
360170000 0.381047395373702
360180000 0.409664718516469
360190000 0.3314533471943869
360200000 0.5201003945430848
360210000 1.320957384981831
360220000 1.3068502883737023
360230000 0.6885962753891349
360240000 0.5117680480977297
360250000 1.0445957011483973
360260000 1.0887115970260568
360270000 0.8510526700386801
360280000 1.053361917104105
360290000 0.6296002845011817
360300000 0.7441197488000393
360310000 0.47907540866147813
360320000 0.9664065833601818
360330000 1.0109928539149762
360340000 0.7069133613716099
3603500

362600000 0.1574639323496355
best so far: 0
type: [2, 1, 26, 2, 1] 104
cases of this type: 5408
best so far: 0
type: [2, 1, 52, 1, 1] 104
cases of this type: 2704
best so far: 0
type: [2, 2, 1, 1, 26] 104
cases of this type: 913952
362610000 0.16188908511173725
362620000 0.4240328838257326
362630000 0.24850245729815296
362640000 0.5210492413362264
362650000 0.4096026106680896
362660000 0.3480127533504433
362670000 0.384801756148994
362680000 0.5808977652391594
362690000 0.42056323623900943
362700000 0.38174375871636473
362710000 0.3580221181065639
362720000 0.30438149528858394
362730000 0.5099176480590105
362740000 0.3513746225534479
362750000 0.2689586063253217
362760000 0.2635724023704529
362770000 0.23319018015958864
362780000 0.30566864958008794
362790000 0.37005303328605493
362800000 0.25427587031086946
362810000 0.19291149505332444
362820000 0.4519436372919878
362830000 0.2779460313451356
362840000 0.20047655034769243
362850000 0.3697468096773029
362860000 0.5156254010812508
3628

364800000 0.059417803740382205
364810000 0.09176326916253567
364820000 0.08235638607111904
364830000 0.11328169176840784
364840000 0.058263912726442026
364850000 0.0776140295938717
364860000 0.05900957904265324
364870000 0.08592896223004659
364880000 0.09378333580360147
364890000 0.05373854111675421
364900000 0.0605005249342786
364910000 0.07044195822300514
364920000 0.06283286177909374
364930000 0.05390884513194694
best so far: 0
type: [4, 1, 1, 2, 13] 104
cases of this type: 228488
364940000 0.04747023737561702
364950000 0.07454392096080383
364960000 0.05770082623784409
364970000 0.04590344218016995
364980000 0.04384993002287547
364990000 0.03859478561079502
365000000 0.05018179847007989
365010000 0.05951949474215507
365020000 0.04587167729247279
365030000 0.019688358282453484
365040000 0.025026272036770977
365050000 0.02952740912853347
365060000 0.02034444809832838
365070000 0.026732182659526666
365080000 0.023787501546872987
365090000 0.01825756445030371
365100000 0.020850655728220